In [ ]:
import pandas as pd
import numpy as np
import re
import glob
import os
import datetime
import matplotlib.pyplot as plt
import itertools

### env variables

In [ ]:
BASE_PATH = '../data/pcs-scraping'
RESULTS_PATH = '../data/pcs-scraping/results/rider'
RANKINGS_PATH = '../data/pcs-scraping/pcs-rankings/rider'
TEAMS_PATH = '../data/pcs-scraping/teams/rider'
CALENDARS_PATH = '../data/pcs-scraping/calendars'
STARTLISTS_PATH = '../data/pcs-scraping/startlists'
RACERESULTS_PATH = '../data/pcs-scraping/race_results'

### Load data

In [ ]:
rider_names = list(pd.read_csv('../data/pcs-scraping/rider_names.csv'))

In [ ]:
rider_name = rider_names[0]

df_rider = {rider_name: {}}

for file in glob.glob(os.path.join(RESULTS_PATH, rider_name, '*.csv')):
    
    year = re.search('\d{4}', file).group(0)
    df_rider[rider_name][year] = pd.read_csv(file)


In [ ]:
df_rider[rider_name]['2021']

# Expert model

- competitor with most victories/top X in the last N races is predicted as race winner

### Data Cleaning

In [ ]:
def clean_data(rider_names, year):
    
    df_season = {year: {}}
    df_res = {}
    
    for rider_name in rider_names:
        for file in glob.glob(os.path.join(RESULTS_PATH, rider_name, f'{year}.csv')):

            df_season[year][rider_name] = pd.read_csv(file)

            # Filter NaN values and non int
            df_results = df_season[year][rider_name][['Date', 'Result', 'Race']].dropna(axis=0)
            df_results = df_results[(df_results.Result != 'DNF') & (df_results.Result != 'DNS') & (df_results.Result != 'DSQ') & (df_results.Result != 'OTL')]
            results = [int(x) for x in df_results.Result]

            df_results['Result'] = results
            df_res[rider_name] = df_results
    
    return df_res

In [ ]:
year = '2021'
df_res = clean_data(rider_names[:100], year)

### Compute Model Features (Win Statistics)

In [ ]:
def GetStats(df):
    
    stats = {}
    
    stats['wins'] = (df.Result == 1).sum()
    stats['top_3'] = (df.Result <= 3).sum()
    stats['top_5'] = (df.Result <= 5).sum()
    stats['top_10'] = (df.Result <= 10).sum()
    stats['top_20'] = (df.Result <= 20).sum()
    
    return stats

In [ ]:
year = '2021'
stats = {year: {}}

for rider_name in rider_names[:10]:
    
    stats[year][rider_name] = GetStats(df_res[rider_name])

In [ ]:
stats['2021']['adam-yates']

### Rename result column with rider name

In [ ]:
for rider_name in rider_names[:10]:
    
    df_res[rider_name].rename(columns={'Result': f'Result-{rider_name}'}, inplace=True)

### Merge dataframes

In [ ]:
pd.merge(df_res['wout-van-aert'], df_res['mathieu-van-der-poel'], on='Race')

 # Compute Features for each participant at race

### Get race startlist

In [ ]:
startlist = pd.read_csv(os.path.join(STARTLISTS_PATH, 'giro-d-italia', '2021', 'startlist.csv'))
startlist.head()

### Normalize startlist names

In [ ]:
def normalize_name(name):
    
    norm_name = '-'.join(reversed(name.lower().replace(' ', '-').split('-')))
    
    return norm_name

In [ ]:
startlist_norm = [normalize_name(name) for name in startlist.Ridername]
#startlist_norm

### Get competing riders

In [ ]:
riders_competing = []

for rider_name in rider_names[:100]:
    if rider_name in startlist_norm:
        riders_competing.append(rider_name)
        
riders_competing

### Data normalization functions

In [ ]:
def convert_to_datetime(date_string, year):
    
    if not isinstance(date_string, datetime.datetime):
        return datetime.datetime.strptime(date_string, '%d.%m').replace(year=year)
    else:
        return date_string

def normalize_date_data(riders_competing):
    
    df = {k: v for k, v in df_res.items() if k in riders_competing}
    
    for rider in riders_competing:
        datetimes = [convert_to_datetime(date, int(year)) for date in df[rider].Date]
        df[rider].Date = datetimes
    
    return df

def create_date_mask(df, event, year, timedelta, riders_competing):
    
    calendar = pd.read_csv(os.path.join(CALENDARS_PATH, f'{year}.csv'))
    event_date = calendar[calendar.Race.str.contains(event)].Date.values[0].split('- ')[-1]
    
    event_date = datetime.datetime.strptime(f'{event_date}.{year}', '%d.%m.%Y')
    start_date = event_date - datetime.timedelta(weeks=timedelta)
    
    for rider in riders_competing:
        mask = (df[rider].Date > start_date) & (df[rider].Date <= event_date)
        df[rider] = df[rider].loc[mask]

    return df

### Prediction functions

In [ ]:
def compute_features(riders_competing, event, year, timedelta):
    
    df = normalize_date_data(riders_competing)
        
    df_mask = create_date_mask(df, event, year, timedelta, riders_competing)
    
    stats = {}
    for rider in riders_competing:
        stats[rider] = GetStats(df_mask[rider])
        
    return stats

def get_score(riders_competing, weights):
    
    score = {}
    for rider in riders_competing:
        score[rider] = [np.average(np.array(list(features[rider].values())), weights=weights)]
        
    return score

def predict_winner(score):
    
    return max(score, key=score.get)

def get_gc_winner(gc):
    
    return "-".join(reversed(gc.Rider.iloc[0].replace(gc.Team.iloc[0], "").split(" "))).lower()

In [ ]:
features = compute_features(riders_competing, "Giro d'Italia", '2021', 12)

In [ ]:
weights = np.array([10, 8, 6, 4, 2])
score = get_score(riders_competing, weights)
pred = predict_winner(score)
pred

In [ ]:
gc = pd.read_csv(os.path.join(RACERESULTS_PATH, 'giro-d-italia', year, 'gc.csv'))
y = get_gc_winner(gc)
y

### Plot scores

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(14, 14))
pd.DataFrame(features).plot(kind='bar', ax=ax1, colormap='tab20c')
pd.DataFrame(score).plot(kind='bar', ax=ax2, colormap='tab20c')

ax1.set_ylabel('Count')
ax2.set_ylabel('Score')
ax2.set_xticklabels(["Giro d'Italia 2021"], rotation=0)
ax1.legend(prop={'size': 7})
ax2.legend(prop={'size': 8})

plt.show()

# Model Hyperparams

- timeframe to consider when computing win features (eg. 2 weeks, 2 months, etc.)
- only consider timeframe or number of N races before event ?
- how many (which) win features to include ?
- weighting (linear, exp, etc.) of features to create score ?

### Vary time window

In [ ]:
preds = []
for w in range(1, 40):
    
    year = '2021'
    df_res = clean_data(rider_names[:100], year)

    features = compute_features(riders_competing, "Giro d'Italia", year, w)
    
    weights = np.array([10, 8, 6, 4, 2])
    score = get_score(riders_competing, weights)
    pred = predict_winner(score)
    
    preds.append(pred)

In [ ]:
gc = pd.read_csv(os.path.join(RACERESULTS_PATH, 'giro-d-italia', year, 'gc.csv'))
y = get_gc_winner(gc)

In [ ]:
sum([pred==y for pred in preds])/len(preds)

### Vary prediction features

In [ ]:
def compute_features(riders_competing, event, year, timedelta, selected_stats):
    
    df = normalize_date_data(riders_competing)
        
    df_mask = create_date_mask(df, event, year, timedelta, riders_competing)
    
    stats = {}
    for rider in riders_competing:
        stats[rider] = GetStats(df_mask[rider])
        stats[rider] = {k:v for k, v in stats[rider].items() if k in selected_stats}
        
    return stats

In [ ]:
gc = pd.read_csv(os.path.join(RACERESULTS_PATH, 'giro-d-italia', year, 'gc.csv'))
y = get_gc_winner(gc)

In [ ]:
acc = []
year = '2021'
df_res = clean_data(rider_names[:100], year)

selected_stats = ['wins', 'top_3', 'top_5', 'top_10', 'top_20']

for r in range(1, 6):
    cmb_stats = list(itertools.combinations(selected_stats, r))
    print(cmb_stats)

    preds = []
    for cmb in cmb_stats:

        features = compute_features(riders_competing, "Giro d'Italia", year, w, list(cmb))
        weights = np.ones(len(cmb))
        score = get_score(riders_competing, weights)
        pred = predict_winner(score)

        preds.append(pred)
        
    print(preds)
    acc.append(sum([pred==y for pred in preds])/len(preds))

In [ ]:
acc

# TRAIN - VAL - TEST SPLIT

- create trn, val, test sets
- for a given year, randomly sample results from time window (eg. 12 weeks before target event) for a rider and compute top stats
- OR randomly sample riders and compute top stats for time window

# CROSS-VALIDATION